# Onboarding a Data Scientist

In this notebook you will control the OM Authority agent to issue a credential attesting to another entities ability to participate in a PPML flow as a Data Scientist. 

This notebook and the next one should feel pretty familiar to you, the only difference is that instead of interacting with a mobile agent we will be interacting with the data-scientist agent we ran through docker-compose.

## 1. Initialise your Agent

Standard stuff, we need to do this in every notebook. Although note that in an application you would only do this once and make the controller object accessible throughout the application where needed.

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
from termcolor import colored,cprint
# I think this is because jupyter notebook also runs an event loop
import nest_asyncio
nest_asyncio.apply()
import os

api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=admin_url, api_key=api_key)

# The location the controller spins up a service and listens for webhooks from the agent
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

agent_controller.init_webhook_server(webhook_host=webhook_host, webhook_port=webhook_port)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Start Webhook Server

This starts an aiohttp server at the location specified by the WEBHOOK args above. This is how the controller receives webhooks from the agent, but it is the listeners and their handler functions that define how the application should react to this messages.

In [2]:
loop = asyncio.get_event_loop()
loop.run_until_complete(agent_controller.listen_webhooks())

## 3. Register Listeners for Webhook Topics

This is the same as in notebook 4.

In [3]:

def cred_handler(payload):
    print("Handle Credential Webhook Payload")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange ID {exchange_id}")
    print("Agent Protocol Role", role)
    print("Protocol State ", state )
    if state == "offer_sent":
        print("Credential Offer with the following attributes sent ", attributes)
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    if state == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Subscribing too: issue_credential
Subscribing too: connections


## 4. Fetch schema and definition identifiers

In [4]:
%store -r data_scientist_schema_id
%store -r data_scientist_cred_def_id

if data_scientist_schema_id and data_scientist_cred_def_id:
    print("Successfully loaded identifiers required to issue Data Scientist credentials")
    print("Schema :", data_scientist_schema_id)
    print("Cred Def :", data_scientist_cred_def_id)

Successfully loaded identifiers required to issue Data Scientist credentials
Schema : L2f3UYR1mm2dQHRsx6nX3E:2:OM Data Scientist:0.0.1
Cred Def : E5C5jardyzNBkZwpHsTkiM:3:CL:6667:default


## 5. Populate Credential Attributes

Again you can put whatever you would like in these attributes, but think about how they might be populated if the trusted authority was a research regulator or ethics body.

In [5]:

name=input("Please enter the name for the researcher: ")
scope=input("Please enter the scope of their research: ")
credential_attributes = [
    {"name": "name", "value": name},
    {"name": "scope", "value": scope},
]
print(credential_attributes)

Please enter the name for the researcher:  Wip
Please enter the scope of their research:  Health


[{'name': 'name', 'value': 'Wip'}, {'name': 'scope', 'value': 'Health'}]


## 6. Add Listener to Issue this Credential

Rather than issuing this credential in it's own notebook cell, we will react to connection's becoming active and automatically issue them with the Data Scientist credential.

While automatically issuing credential's to any connection is likely to be frowned upon in realistic scenarios, this illustrates how listeners can be applied to handle more complex application functionality. We will see this used later in the course a lot.

Notice how we can get the connection_id attribute from the webhook payload.

In [6]:
def issue_active_conn(payload):
    connection_id = payload["connection_id"]
    state = payload['state']
    print("State", state)
    if state == 'response':
        # seems to need a sleep 
        time.sleep(1)
        asyncio.get_event_loop().run_until_complete(agent_controller.messaging.trust_ping(connection_id, "hey"))
    elif state == 'active':
        asyncio.get_event_loop().run_until_complete(agent_controller.issuer.send_credential(connection_id, data_scientist_schema_id, data_scientist_cred_def_id, credential_attributes, trace=False))
        

active_conn_listener = {
    "topic": "connections",
    "handler": issue_active_conn
}

agent_controller.add_listener(active_conn_listener)

Subscribing too: connections


## 7. Establish A Connection with the Data Scientist

Like in notebook 4. you will create request the om-authority-agent creates an invitation, however instead of diplaying this as a QR Code you will copy the invitation json across to the Data Scientist notebook labeled 5.

The Data Scientist should be at port 8890, either at localhost if your are running this on your own machine or at the url for your aws instance.

You may need to fetch the token for this jupyter instance using the `./scripts/get_URLS.sh` from the route of PyDentity if you have not done so already.

In [8]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation - Copy The Object Below \n")
print(invite["invitation"])
print("\n\n ----------------------------------------")

Connection ID 17622573-1b72-4199-8684-94dd1b037153
Invitation - Copy The Object Below 

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3c8aadda-3405-4ef9-9968-92c14f016230', 'label': 'OpenMined', 'recipientKeys': ['CJhCVsrLAArEzjKqrbKnP2fquJ7aFCrzkRDQaYWhc9e1'], 'serviceEndpoint': 'https://9b18-86-18-68-143.ngrok.io'}


 ----------------------------------------
Handle Connection Webhook Payload
Connection ID 17622573-1b72-4199-8684-94dd1b037153
State invitation
State invitation
Handle Connection Webhook Payload
Connection ID 17622573-1b72-4199-8684-94dd1b037153
State request
State request
Handle Connection Webhook Payload
Connection ID 17622573-1b72-4199-8684-94dd1b037153
State response
State response
Handle Connection Webhook Payload
Connection ID 17622573-1b72-4199-8684-94dd1b037153
State active
Connection 17622573-1b72-4199-8684-94dd1b037153 changed state to active
State active
Handle Credential Webhook Payload
Credential exchange ID f57d3f13-2b9

## Continue in the Data Scientist Notebook

Be sure to come back here to review the webhook messages your receive after accepting the invitation. Pretty handy how your application reacted to the connection becoming active and immediately offered a credential to this connection.

## End of Notebook

Great, once your data scientist has the credential this notebook is finished. Make sure you terminate the controller instance.

In [9]:
await agent_controller.terminate()

## Continue to Notebook 6.

This notebook is going to be a very similar flow to the one you just ran through here. You will be issuing the data owner a credential instead.